# Projeto 2 - Ciência dos Dados

Professor: Fábio Miranda 


Alunos: Giulia Araujo Castro e Pedro Célia 

# Classificador automático de sentimento

In [55]:
%%capture

#Instalando o tweepy
!pip install tweepy

#Importando as bibliotecas necessárias
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

## Autenticando no  Twitter

* Conta: ***[@giuacastro_]***

In [56]:
#Leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

FileNotFoundError: [Errno 2] No such file or directory: 'auth.pass'

### Escolha de um produto e coleta das mensagens

In [57]:
#Produto escolhido para análise:
produto = 'Harry Potter'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 500

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

In [58]:
#Capturando os dados do twitter 

#Cria um objeto para a captura
#api = tweepy.API(auth)


#Inicia a captura, para mais detalhes: ver a documentação do tweepy
#i = 1
#msgs = []
#for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
#    msgs.append(msg.full_text.lower())
#    i += 1
#    if i > n:
#        break


#Embaralhando as mensagens para reduzir um possível viés
#shuffle(msgs)


In [59]:
#Salvando em uma planilha no Excel

#Verifica se o arquivo não existe para não substituir um conjunto pronto
#if not os.path.isfile('./{0}.xlsx'.format(produto)): 
#    Abre o arquivo para escrita
#    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))


    #divide o conjunto de mensagens em duas planilhas
#    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
#    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

#    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
#    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

#    fecha o arquivo
#    writer.save()

In [60]:
#Criando um set para retirar os tweets repetidos do banco de dados

#dados = pd.read_excel("Harry Potter.xlsx")
#dados2 = set(dados.Treinamento)
#HP = pd.Series(list(dados2))
#HP.to_excel("Harry_Potter.xlsx")

In [61]:
#Limpando as bases obtidas
import re 

def cleanup(text):
    import string
    punctuation = '[!\-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [62]:
#Lendo o excel final obtido
data_raw = pd.read_excel("base_final.xlsx", sheet_name = "Treinamento")
#Nomeando a coluna com as classificações dadas
data_raw = data_raw.rename(columns = {"Unnamed: 1": "Classificação"})

#Acessando os tweets Irrelevantes (0) e os Relevantes (1)
treinamento0 = data_raw.loc[data_raw["Classificação"] == 0]["Treinamento"]
treinamento1 = data_raw.loc[data_raw["Classificação"] == 1]["Treinamento"]

In [63]:
#Criando uma série com os dados gerais e com os tweets já classificados  
Serie_data = pd.Series(data_raw["Treinamento"])
serie_um = pd.Series(treinamento1)
serie_zero = pd.Series(treinamento0)

#Atribuindo o valor 1 para cada tweet 
data_abs = Serie_data.value_counts()
Tserie1 = serie_um.value_counts()
Tserie0 = serie_zero.value_counts()

In [64]:
#Unindo todos os tweets 
trei0 = " ".join(treinamento0)
trei1 = " ".join(treinamento1)

In [65]:
#Limpando os tweets 
zero_trei = cleanup(trei0.lower())
um_trei = cleanup(trei1.lower())

In [66]:
#Dividindo as séries por palavras 
Serie0 = pd.Series(zero_trei.split())
Serie1 = pd.Series(um_trei.split())

In [67]:
#Chegando aos valores relativos de cada palavra 
Tabela0_relativa = Serie0.value_counts(True)
Tabela1_relativa = Serie1.value_counts(True)
Tabela0_abs = Serie0.value_counts()
Tabela1_abs = Serie1.value_counts()

In [68]:
#Obtendo as probabilidades de um tweet ser relevante ou não
Prel = (Tserie1.sum())/(data_abs.sum())
Pirr = (Tserie0.sum())/(data_abs.sum())

In [82]:
d_lista_relev = Tabela1_abs[1]
d_lista_irrelev = Tabela0_abs[1]
d = d_lista_relev+d_lista_irrelev

def probabilidade(palavra):
    if palavra in Tabela1_abs:
        k = Tabela1_abs[palavra]
    else:
        k = 0
    if palavra in Tabela0_abs:
        J = Tabela0_abs[palavra]
    else:
        J = 0
    prob_relev =( k +1)/(len(Serie1) + d)
    prob_irrelev =(J + 1)/(len(Serie0) + d) 
    return [prob_relev, prob_irrelev]

### Montando o Classificador Naive-Bayes

In [83]:
#Lendo a tabela obtida
data_teste = pd.read_excel("base_final.xlsx", sheet_name = "Teste")
#Nomeando a coluna com as classificações
data_teste = data_teste.rename(columns = {"Unnamed: 1": "Classificação"})

In [84]:
#Limpando os tweets de teste

#Cria uma lista para adicionar os tweets limpos
tweets_limpos = []
#Chama a função cleanup
for tweet in data_teste["Teste"]:
    tweets_limpos.append(cleanup(tweet)) 

In [85]:
#Classificador 

#Lista para adicionarmos a classificação
classificacao = []

for frase in tweets_limpos:
    #Separando o tweet em palavras
    frasesp = frase.split()
    #Cria uma lista de palavras para cada tweet
    lista_palavras = []
    
    for palavra in frasesp:
        #Adicionando as palavras na lista de palavras
        lista_palavras.append(palavra)
        #Criando as probabilidades de ser Relevante (Pr) ou Irrelevante (Pir)
        Pr = 1
        Pir = 1
        
        for pal in lista_palavras:
            
            #Se a palavra estiver presente nas duas tabelas
            if pal in Tabela1_relativa and pal in Tabela0_relativa:
                Pr *= Tabela1_relativa[pal]
                Pir *= Tabela0_relativa[pal]
            
            #Se estiver apenas na tabela Irrelevante
            elif pal in Tabela0_relativa:
                Pir *= Tabela0_relativa[pal]
                probabilidade(pal)[0]*Pr
            
            #Se estiver apenas na tabela Relevante
            elif pal in Tabela1_relativa:
                Pr *= Tabela1_relativa[pal]
                probabilidade(pal)[1]*Pir
                
            #Se não estiver em nenhuma     
            else:
                probabilidade(pal)[0]*Pr
                probabilidade(pal)[1]*Pir
    
    #Calcula a probablidade de ser Relevante ou Irrelevante
    P_irrelevante = Pir * Pirr
    P_relevante = Pr * Prel
    
    #Compara os valores obtidos e adiciona a classificação na lista 
    if P_irrelevante > P_relevante: 
        classificacao.append(0)
    elif P_relevante > P_irrelevante:
        classificacao.append(1)
    else: 
        classificacao.append("none")

### Verificando a performance

In [87]:
#Transformando a tabela de classificação feita à mão em uma lista
x = data_teste["Classificação"]
lista = []
for e in x: 
    lista.append(e)

In [88]:
#Comparando as classificações
i = 0
e = 0
while i < len(lista): 
    if lista[i] == classificacao[i]:
        #Obtendo o número de classificações corretas do classificador
        e+=1
    i+=1
print(e)

89


In [89]:
#Porcentagem de acertos obtidos 
P = e/220

P*100

40.45454545454545